In [1]:
### STEP 1. Preamble
import coloredlogs
import ipdb
import json
import logging
import os
import pylatex
import pylatexenc
from pylatexenc import latexencode  # line is necessary
import re
import subprocess
import tkinter
import tkinter.filedialog

verbose = True

### STEP 2. Set up logger
log = logging.getLogger(__name__)
if verbose:
    coloredlogs.install(fmt='%(asctime)s [%(levelname)s] %(message)s', level=logging.DEBUG, logger=log)
else:
    coloredlogs.install(fmt='%(asctime)s [%(levelname)s] %(message)s', level=logging.INFO, logger=log)

In [2]:
### STEP 3. Read JSON infile
tkinter.Tk().withdraw() ## Prevent root window
in_fn = tkinter.filedialog.askopenfilename()

output_fn = in_fn.strip(".json")

with open(in_fn, 'r') as file:
    masterDict = json.load(file)   